In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
dna_CC11_PD_grp_prepared = dataiku.Dataset("DNA_CC11_PD_grp_prepared")
dna_CC11_PD_grp_prepared_df = dna_CC11_PD_grp_prepared.get_dataframe()

import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings("ignore")
plt.style.use('Solarize_Light2')

#df = pd.DataFrame(dna_CC11_PD_grp_prepared_df, columns=['date','ID_count'])
df = pd.DataFrame(dna_CC11_PD_grp_prepared_df)

In [0]:
#df.columns
#df.date.max()

In [0]:
from datetime import datetime as dt
days = pd.date_range('2011-01-01', periods=111, freq='M')

df_new=pd.DataFrame()

df_new['Date_2'] = pd.to_datetime(days)
df_new['val']=0
df_new['year_j']=pd.DatetimeIndex(df_new['Date_2']).year
df_new['month_j']=pd.DatetimeIndex(df_new['Date_2']).month
#df_new.head()

In [0]:
#df_new.info()
#df_new.Date_2.max()

In [0]:
df_fin= pd.merge(df_new, df, how='left',left_on=['year_j','month_j'], right_on=['Sign up year', 'Sign up Month'])

In [0]:
df_fin=df_fin.fillna(0)
df_fin['data']=df_fin['val']+df_fin['ID_count']
df_fin=df_fin[['Date_2','data']]
#df_fin.head()

In [0]:
df=df_fin.copy()
df.columns=['time','data']
df = df.set_index(['time'])

In [0]:
df=df.asfreq('M')
#df.index

In [0]:
train = df.iloc[:-10, :]
test = df.iloc[-10:, :]
pred = test.copy()
df.plot(figsize=(12,3))
print("      Monthwise SignRate for C11 for Course Category Personal Development");

In [0]:
df['z_data'] = (df['data'] - df.data.rolling(window=12).mean()) / df.data.rolling(window=12).std()
df['zp_data'] = df['z_data'] - df['z_data'].shift(12)

fig, ax = plt.subplots(3,figsize=(12, 9))
ax[0].plot(df.index, df.data, label='raw data')
ax[0].plot(df.data.rolling(window=12).mean(), label="rolling mean");
ax[0].plot(df.data.rolling(window=12).std(), label="rolling std (x10)");
ax[0].legend()
ax[0].set_title("Analyzing the Sign-up Rate for trend and lags");

ax[1].plot(df.index, df.z_data, label="de-trended data")
ax[1].plot(df.z_data.rolling(window=12).mean(), label="rolling mean");
ax[1].plot(df.z_data.rolling(window=12).std(), label="rolling std (x10)");
ax[1].legend()
ax[1].set_title("Analyzing the detrended Sign-up Rate for lags");

ax[2].plot(df.index, df.zp_data, label="12 lag differenced de-trended data")
ax[2].plot(df.zp_data.rolling(window=12).mean(), label="rolling mean");
ax[2].plot(df.zp_data.rolling(window=12).std(), label="rolling std (x10)");
ax[2].legend()
ax[2].set_title("Analyzing the lagged SignRate trend for Stationarity")
plt.tight_layout()
fig.autofmt_xdate()

In [0]:
from statsmodels.tsa.stattools import adfuller

print(" > Is the data stationary ?")
dftest = adfuller(df.data, autolag='AIC')
print("Test statistic = {:.3f}".format(dftest[0]))
print("P-value = {:.3f}".format(dftest[1]))
print("Critical values :")
for k, v in dftest[4].items():
    print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<dftest[0] else "", 100-int(k[:-1])))

print("\n > Is the de-trended data stationary ?")
dftest = adfuller(df.z_data.dropna(), autolag='AIC')
print("Test statistic = {:.3f}".format(dftest[0]))
print("P-value = {:.3f}".format(dftest[1]))
print("Critical values :")
for k, v in dftest[4].items():
    print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<dftest[0] else "", 100-int(k[:-1])))

print("\n > Is the 12-lag differenced de-trended data stationary ?")
dftest = adfuller(df.zp_data.dropna(), autolag='AIC')
print("Test statistic = {:.3f}".format(dftest[0]))
print("P-value = {:.3f}".format(dftest[1]))
print("Critical values :")
for k, v in dftest[4].items():
    print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<dftest[0] else "", 100-int(k[:-1])))

In [0]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2, figsize=(12,6))
ax[0] = plot_acf(df.z_data.dropna(), ax=ax[0], lags=20)
ax[1] = plot_pacf(df.z_data.dropna(), ax=ax[1], lags=20)

In [0]:
df=df_fin.copy()
df.columns=['time','data']
df = df.set_index(['time'])
df=df.asfreq('M')
train = df.iloc[:-10, :]
test = df.iloc[-10:, :]
pred = test.copy()

In [0]:
train.index = pd.to_datetime(train.index)
test.index = pd.to_datetime(test.index)
pred = test.copy()

In [0]:
print('Train Data Information')
train.info()

In [0]:
print('Test Data Information')
test.info()

In [0]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
plt.style.use('Solarize_Light2')


model = SimpleExpSmoothing(np.asarray(train['data']))
model._index = pd.to_datetime(train.index)

fit1 = model.fit()
pred1 = fit1.forecast(10)
fit2 = model.fit(smoothing_level=.2)
pred2 = fit2.forecast(10)
fit3 = model.fit(smoothing_level=.5)
pred3 = fit3.forecast(10)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(train.index[50:], train.values[50:])
ax.plot(test.index, test.values, color="gray")
for p, f, c in zip((pred1, pred2, pred3),(fit1, fit2, fit3),('#ff7823','#3c763d','c')):
    ax.plot(train.index[50:], f.fittedvalues[50:], color=c)
    ax.plot(test.index, p, label="alpha="+str(f.params['smoothing_level'])[:3], color=c)
plt.title("Simple Exponential Smoothing")
plt.legend();

model = Holt(np.asarray(train['data']))
model._index = pd.to_datetime(train.index)

fit1 = model.fit(smoothing_level=.3, smoothing_slope=.05)
pred1 = fit1.forecast(10)
fit2 = model.fit(optimized=True)
pred2 = fit2.forecast(10)
fit3 = model.fit(smoothing_level=.3, smoothing_slope=.2)
pred3 = fit3.forecast(10)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(train.index[50:], train.values[50:])
ax.plot(test.index, test.values, color="gray")
for p, f, c in zip((pred1, pred2, pred3),(fit1, fit2, fit3),('#ff7823','#3c763d','c')):
    ax.plot(train.index[50:], f.fittedvalues[50:], color=c)
    ax.plot(test.index, p, label="alpha="+str(f.params['smoothing_level'])[:4]+", beta="+str(f.params['smoothing_slope'])[:4], color=c)
plt.title("Holt's Exponential Smoothing")
plt.legend();

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df['data'])

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(df['data'], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [0]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np

model = ExponentialSmoothing(np.asarray(train['data']), trend='add', seasonal=None)
model2 = ExponentialSmoothing(np.asarray(train['data']), trend='add', seasonal=None, damped=True)
model._index = pd.to_datetime(train.index)

fit1 = model.fit()
fit2 = model2.fit()
pred1 = fit1.forecast(10)
pred2 = fit2.forecast(10)

fig, ax = plt.subplots(2, figsize=(12, 12))
ax[0].plot(train.index[50:], train.values[50:])
ax[0].plot(test.index, test.values, color="gray", label="truth")
ax[1].plot(train.index[50:], train.values[50:])
ax[1].plot(test.index, test.values, color="gray", label="truth")
for p, f, c in zip((pred1, pred2),(fit1, fit2),('#ff7823','#3c763d')):
    ax[0].plot(train.index[50:], f.fittedvalues[50:], color=c)
    ax[1].plot(train.index[50:], f.fittedvalues[50:], color=c)
    ax[0].plot(test.index, p, label="alpha="+str(f.params['smoothing_level'])[:4]+", beta="+str(f.params['smoothing_slope'])[:4]+ ", damping="+str(True if f.params['damping_slope']>0 else False), color=c)
    ax[1].plot(test.index, p, label="alpha="+str(f.params['smoothing_level'])[:4]+", beta="+str(f.params['smoothing_slope'])[:4]+ ", damping="+str(True if f.params['damping_slope']>0 else False), color=c)
ax[0].set_title("Damped Exponential Smoothing");
ax[1].set_title("Damped Exponential Smoothing - zoomed");
plt.legend();

In [0]:
fig, ax = plt.subplots(2, sharex=True, figsize=(12,6))
ax[0].plot(df.data.values);
ax[0].set_title("Raw data");
ax[1].plot(np.log(df.data.values));
ax[1].set_title("Logged data (deflated)");
ax[1].set_ylim(0, 15);

fig, ax = plt.subplots(2, 2, figsize=(12,6))
first_diff = (np.log(df.data)- np.log(df.data).shift()).dropna()
ax[0, 0] = plot_acf(np.log(df.data), ax=ax[0, 0], lags=20, title="ACF - Logged data")
ax[1, 0] = plot_pacf(np.log(df.data), ax=ax[1, 0], lags=20, title="PACF - Logged data")
ax[0, 1] = plot_acf(first_diff , ax=ax[0, 1], lags=20, title="ACF - Differenced Logged data")
ax[1, 1] = plot_pacf(first_diff, ax=ax[1, 1], lags=20, title="PACF - Differenced Logged data")

In [0]:
from statsmodels.tsa.stattools import kpss

print(" > Is the data stationary ?")
dftest = kpss(np.log(df.data), 'ct')
print("Test statistic = {:.3f}".format(dftest[0]))
print("P-value = {:.3f}".format(dftest[1]))
print("Critical values :")
for k, v in dftest[3].items():
    print("\t{}: {}".format(k, v))

In [0]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(df.data.dropna(), (1, 0, 1))
res_000 = model.fit()
print(res_000.summary())

model = ARIMA(df.data.dropna(), (1, 0, 0))
res_010 = model.fit()
print(res_010.summary())

fig, ax = plt.subplots(1, 2, sharey=True, figsize=(12, 6))
ax[0].plot(res_000.resid.values, alpha=0.7, label='variance={:.3f}'.format(np.std(res_000.resid.values)));
ax[0].hlines(0, xmin=0, xmax=350, color='r');
ax[0].set_title("ARIMA(1,0,1)");
ax[0].legend();
ax[1].plot(res_010.resid.values, alpha=0.7, label='variance={:.3f}'.format(np.std(res_010.resid.values)));
ax[1].hlines(0, xmin=0, xmax=350, color='r');
ax[1].set_title("ARIMA(1,0,0)");
ax[1].legend();

In [0]:
model = ARIMA(train.data.dropna(), (1, 0, 1))
res_101 = model.fit()
pred_ARIMA = res_101.forecast(10)

fig, ax = plt.subplots(figsize=(12, 6))
df.index = pd.to_datetime(df.index, format="%Y-%m")
df.data.dropna()[50:].plot(ax=ax);
ax.vlines('2019-06', 13, 14.5, linestyle='--', color='r', label='Start of forecast');

# - NOTE from the official documentation :
# -- The dynamic keyword affects in-sample prediction.
# -- If dynamic is False, then the in-sample lagged values are used for prediction.
# -- If dynamic is True, then in-sample forecasts are used in place of lagged dependent variables.
ax = res_101.plot_predict('2019-06', '2020-12', dynamic=True, plot_insample=False, ax=ax);

In [0]:
pred_ARIMA = res_101.predict(start='2019-06-30', end='2020-03-31')
#pred_ARIMA

In [0]:
import math
#MAPE
y_true=test.data.values
y_pred=pred_ARIMA.values

forecast_error = y_true-y_pred

mean_forecast_error = np.mean(forecast_error)
mean_absolute_error = np.mean( np.abs(forecast_error) )
mean_squared_error = np.mean(forecast_error**2)
rmse = math.sqrt(mean_squared_error)
MAPE=np.mean(np.abs((y_true-y_pred)/y_true))*100
print('mean_forecast_error= %f'%mean_forecast_error)
print('mean_absolute_error= %f'%mean_absolute_error)
print('mean_squared_error= %f'%mean_squared_error)
print('rmse= %f'%rmse)
print('MAPE= %f'%MAPE)

In [0]:
time_series_modeled=df.copy()
time_series_modeled['pred']= res_101.predict(start='2011-01-31', end='2020-03-31').values

In [0]:
print('Signup Rate prediction Values for C11 and Personal Development')
time_series_modeled.tail()

In [0]:
Model_df = model # For this sample code, simply copy input to output
time_series_modeled_df=time_series_modeled

# Write recipe outputs

time_series_modeled = dataiku.Dataset("time_series_modeled")
time_series_modeled.write_with_schema(time_series_modeled_df)